In [ ]:
from alefuncs import *

In [ ]:
from subprocess import run
import pexpect
from getpass import getpass

def ssl_encrypt(infile, outfile):
    '''(file_path, file_path) => encrypted_file
    Uses openssl to encrypt/decrypt files.
    '''
    pwd = getpass('enter encryption pwd:')
    if getpass('repeat pwd:') == pwd:
        run(f'openssl enc -aes-256-cbc -a -salt -pass pass:{pwd} -in {infile} -out {outfile}',shell=True)
    else:
        print("passwords don't match.")

    
def ssl_decrypt(infile, outfile):
    '''(file_path, file_path) => decrypted_file
    Uses openssl to encrypt/decrypt files.
    '''
    pwd = getpass('enter decryption pwd:')
    run(f'openssl enc -d -aes-256-cbc -a -pass pass:{pwd} -in {infile} -out {outfile}', shell=True)

In [ ]:
infile = 'alefuncs.py'
outfile = 'alefuncs.enc'
print(f'openssl aes-256-cbc -a -salt -in {infile} -out {outfile} -base64')

infile = 'alefuncs.enc'
outfile = 'alefuncs.dec.py'
print(f'openssl enc -d -aes-256-cbc -in {infile} -out {outfile}')

In [ ]:
ssl_encrypt('alefuncs.py','alefuncs.enc')
ssl_decrypt('alefuncs.enc', 'a2.py')

In [ ]:
run(f'openssl enc -aes-256-cbc -pass  -p -in alefuncs.py -out alefuncs.enc -base64',shell=True)


In [ ]:
import socket   #for sockets
import time

class Server:
    def __init__(self):
        self.id = 'server.007'
        self.ssocket = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
        self.host = 'localhost'
        self.port = 7899
        
    def start(self):
        start_time = time.time()
        print(f'{self.id} started')
        self.ssocket.bind((self.host, self.port))
        self.ssocket.listen(5) # queue up to 5 requests
        
        try:
            while True:
                # establish a connection
                clientsocket, addr = self.ssocket.accept()      
                print(f'Got a connection from {addr}')
                currentTime = time.ctime(time.time()) + "\r\n"
                clientsocket.send(currentTime.encode('ascii'))
                clientsocket.close()                  
        except KeyboardInterrupt:
            return self.terminate()
        
        
            
    def terminate(self):
        print(f'{self.id} terminated')
        self.ssocket.close()
        

In [ ]:
s = Server()
s.start()

In [ ]:
s.host

In [ ]:
import asyncore

class Handler(asyncore.dispatcher):
    def __init__(self, sock):
        self.buffer = b''
        super().__init__(sock)

    def handle_read(self):
        self.buffer += self.recv(1024)
        print(f'current buffer: {repr(self.buffer)}')


class Server(asyncore.dispatcher):
    def __init__(self, host, port):
        asyncore.dispatcher.__init__(self)
        self.create_socket()
        self.set_reuse_addr()
        self.bind((host, port))
        self.listen(5)

    def handle_accepted(self, sock, addr):
        print(f'Incoming connection from {repr(addr)}')
        Handler(sock)


if __name__ == "__main__":
    server = Server("localhost", 1234)
    asyncore.loop()

In [6]:
import asyncore, time

class Client(asyncore.dispatcher):

    def __init__(self, host, port):
        asyncore.dispatcher.__init__(self)
        self.create_socket()
        self.connect( (host, port) )
        self.buffer = bytes("hello world", 'ascii')
        self.last_sent = time.time()

    def handle_connect(self):
        pass

    def handle_close(self):
        self.close()

    def handle_read(self):
        print('Re: ', self.recv(1024))

    def writable(self):
        return (len(self.buffer) > 0)

    def writable(self):
        return True

    def handle_write(self):
        sent = self.send(self.buffer)
        if sent:
            print('Sent:', sent)
            self.buffer = self.buffer[sent:]
            self.last_sent = time.time()
        else:
            if time.time() - self.last_sent > 10:
                self.buffer = 'beat'.encode('utf-8')


client = Client('localhost', 1234)
asyncore.loop()

Sent: 11
Sent: 4
Sent: 4
Sent: 4
Sent: 4
Sent: 4


KeyboardInterrupt: 